In [99]:
import pandas as pd
import glob

In [106]:
data = pd.read_csv('../backup/data/predictions/netrisk_casco_v40_all_predictions.csv', index_col='unique_id')
signal_data = pd.read_parquet('../../singal_dataset/input/DATA_preModel_X2024.07.28_2024-08-06.parquet')
signal_data = signal_data[signal_data['Product_Casco'] == 1]
signal_data = signal_data[['policyNr', 'Deductible_MOD_Factor', 'Deductible_MOD_Abs']].rename(
    columns ={'Deductible_MOD_Factor' : 'DeductiblePercentage', 'Deductible_MOD_Abs' : 'DeductibleAmount'})
data = data[[col for col in data.columns if col.endswith('_price_model')]].reset_index()
data = pd.merge(data.reset_index(), signal_data, left_on='unique_id', right_on = 'policyNr')
data['DeductiblePercentage'] = data['DeductiblePercentage'] * 100
deductibles = pd.read_excel('../../../Downloads/deductible_effect (1).xlsx')
deductibles = deductibles.rename(columns = lambda x : x + '_factor' if x[0] != 'D' else x)
data = pd.merge(data, deductibles, on = ['DeductiblePercentage', 'DeductibleAmount'])

In [107]:
for comp in data.filter(like='factor').columns:
    player = comp.split('_')[0]
    if 'K&AMP&;' in comp:
        price_col = 'netrisk_casco_v36_K&AMP;H_price_model'   
    else:
        price_col = f'netrisk_casco_v36_{player}_price_model'
    if price_col in data.columns:
        data[price_col] = data[price_col] * data[comp].fillna(1)

In [108]:
data = data[['policyNr'] + [col for col in data.columns if col.endswith('_price_model')]]

In [109]:
data.to_csv('signal_casco_netrisk_casco_v36_model_predictions_with_deductibles.csv',)